In [2]:
import pandas as pd
df=pd.read_csv('/content/Housing.csv')

In [3]:
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
df = pd.get_dummies(df, drop_first=True)

In [12]:
df.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking',
       'mainroad_yes', 'guestroom_yes', 'basement_yes', 'hotwaterheating_yes',
       'airconditioning_yes', 'prefarea_yes',
       'furnishingstatus_semi-furnished', 'furnishingstatus_unfurnished'],
      dtype='object')

In [8]:
X = df_encoded.drop(columns=['price'],axis=1)
y = df_encoded['price']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
 ohe =OneHotEncoder(sparse_output = False, drop='first', handle_unknown='ignore')
ct = make_column_transformer((ohe, make_column_selector(dtype_include=object)), remainder='passthrough')


# **XGB REGRESSOR**

In [25]:
from xgboost import XGBRegressor
xgb= XGBRegressor(random_state=24,probability=True )
kfold= StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pipe= Pipeline([('ct',ct),('xgb',xgb)])
pipe.fit(X_train,y_train)
params= {
    'xgb__learning_rate': [0.001, 0.3],
    'xgb__max_depth' :[2,5],
    'xgb__n_estimators' :[10,20]
}
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='r2', verbose=1)
gcv.fit(X, y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('ct',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7806426b1f90>)])),
                                       ('xgb',
                                        XGBRegre...
                                                     max_cat_to_onehot=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     probability=True, ...))]),
             param_grid={'xgb__learning_rate': [0.001, 0.3],
                         'xgb__max_depth': [2, 5],
                         'xgb__n_estimators': [10, 20]},
             scoring='r2', verbose=1)

In [26]:
print(gcv.best_params_)

{'xgb__learning_rate': 0.3, 'xgb__max_depth': 2, 'xgb__n_estimators': 20}


In [27]:
print(gcv.best_score_)

0.6214819073677063


DECISIONTREE REGRESSOR

In [28]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [29]:
# Initialize the Decision Tree Regressor
regressor = DecisionTreeRegressor(random_state=42)

In [31]:
# Train the model
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [32]:
# Make predictions
y_pred = regressor.predict(X_test)

In [37]:
# Initialize the Decision Tree Regressor
rgr = DecisionTreeRegressor(random_state=42)
kfold= StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pipe= Pipeline([('ct',ct),('rgr',rgr)])
rgr.fit(X_train, y_train)
params = {'rgr__max_depth': [3, None],
          'rgr__min_samples_split':[2, 5, 10],
          'rgr__min_samples_leaf':[1, 5]}
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='r2', verbose=1)
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
{'rgr__max_depth': None, 'rgr__min_samples_leaf': 5, 'rgr__min_samples_split': 2}
0.47843364115669595


RANDOMFOREST REGRESSOR

In [63]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd

# Initialize the Decision Tree Regressor
rfr= RandomForestRegressor( random_state=24)
kfold= StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pipe= Pipeline([('ct',ct),('rfr',rfr)])
rfr.fit(X_train, y_train)
params = {'rfr__max_features': [3, 4,5 ]}
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='r2', verbose=1)
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
{'rfr__max_features': 4}
0.6347049310591647


TypeError: '>' not supported between instances of 'DecisionTreeRegressor' and 'DecisionTreeRegressor'

QUESTION NUMBER 2

In [58]:
import pandas as pd
df=pd.read_csv('/content/Glass.csv')

In [67]:
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,building_windows_float_processed
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,headlamps
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,headlamps
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,headlamps
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,headlamps


In [70]:
X = df.drop('Type',axis=1)
y = df['Type']

In [71]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [73]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


knn = KNeighborsClassifier(n_neighbors= 3, metric='manhattan')
kfold= StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
pipe= Pipeline([('ct',ct),('knn',knn)])
pipe.fit(X_train,y_train)
# Step 5: Make Predictions
y_pred = knn.predict(X_test)
# Step 6: Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.6461538461538462